# Test-time dropout script

The goal is to develop a function with a command line interface that takes a trained model with dropout and returns an ensemble prediction, so I imagine something like:

```
python create_dropout_ensemble.py --exp_id 44-resnet_deeper2 --members 100 ...
```

The script should return and save a xarray dataset just like `create_prediction` but with an added dimension `ens_member`.

You basically already did the work in the starter exercise I gave you. You can also check out my solution. Now it's just a matter of creating a convenient script. For examples of command line scripts I wrote, check out `src/extract_level.py` using `argparse` or `scripts/download_tigge.py` using Google's `fire`. Also, see whether your or my method of implementing the test-time dropout is more convenient. Whatever requires fewer changes to the rest of the code (probably yours).

As mentioned in the WeatherBench paper, testing is done using the years 2017 and 2018. This means the ensemble predictions also have to be created for these two years. The data can be downloaded here: https://mediatum.ub.tum.de/1524895. However, the files, which contain all years, are quite large, so you probably don't want to download it to your laptop. I uploaded just the last two years for each variable here: To come...

Next, you need a trained model. I number my experiments (see Dropbox document). You can find two different models in the link above. 

As mentioned in the Dropbox document, I would suggest developing the main function in the notebook. Once that works, you can create a CLI around it and save the script. 

Also, let's use `tensorflow>=2.0`.

To Do:
get data for 2017, 18 in test/. right now just for 17
python run_ensemble.py --input dg_test --exp_id resent --members 100
    

In [1]:
# Here is a useful tip: Using autoreload allows you to make changes to an imported module
# which are then automatically updated in this notebook. This is how I start all my notebooks.
%load_ext autoreload
%autoreload 2 # Every two seconds

In [2]:
from fire import Fire
#!pip install xesmf
import argparse
import xarray as xr
import numpy as np
import xesmf as xe
from glob import glob
import os
from src.data_generator import *
from src.train import *

/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will

In [15]:
args = load_args('../nn_configs/B/54-resnet_d2_new_levs.yml')

In [22]:
args['datadir']= '/Users/sagar/Library/Mobile\ Documents/com~apple~CloudDocs/studies/WeatherBench/Testing_WeatherBench/5.625deg/'
#args['model_save_dir']= '/Users/sagar/Library/Mobile\ Documents/com~apple~CloudDocs/studies/WeatherBench/Testing_WeatherBench/predictions/saved_models/'
args['pred_save_dir']= '/Users/sagar/Library/Mobile\ Documents/com~apple~CloudDocs/studies/WeatherBench/Testing_WeatherBench/predictions/'
args['pretrained_model']='/Users/sagar/Library/Mobile\ Documents/com~apple~CloudDocs/studies/WeatherBench/Testing_WeatherBench/predictions/saved_models/'

In [19]:
args['test_years'] = ['2017', '2018']

5

In [4]:
def main(dg_test, network_type, model_id, forecast_number):
    '''
    #inputs needed: dg_test (should we instead use a numpy array X?),filters, kernels, nt_in, bn_position, use_bias, skip, dropout
    #(for unet: input_shape, n_layers, filters_start, channels_out, kernel, u_ski, res_skip, 
    #directory of model weights
    #output directory
    #forecast_number: number of forecasts per input time.
    '''
    
    args = load_args('../nn_configs/B/54-resnet_d2_new_levs.yml')
    
    #check for network_type (see networks.py), build network
    if network_type == 'resnet':
            model = build_resnet(
                filters, kernels, input_shape=(
            len(dg_test.data.lat), len(dg_test.data.lon), len(dg_test.data.level) * nt_in),
                bn_position=bn_position, use_bias=use_bias, l2=l2, skip=skip,
                dropout=dropout
            )
    elif network_type == 'unet_google':
            model = build_unet_google(
                filters,
                input_shape=(len(dg_test.data.lat), len(dg_test.data.lon),len(dg_test.data.level) * nt_in),
                output_channels=len(dg_test.output_idxs),
                dropout=dropout
            )
#     elif network_type == 'unet': #dont know if this works.
#             model = build_unet(input_shape, n_layers, filters_start, channels_out, kernel=3, u_skip=True,
#                res_skip=True, l2=0, bn_position=None, dropout=0):